Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS104.N11

LAB01 - 19/09/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1mKGw5kBbkB0Tc0mtS2HA1Fkf-yvykbxu?usp=sharing)

# **GraphFrame**

## Installing GraphFrame & PySpark 

In [ ]:
!pip install pyspark==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.3 MB 15 kB/s 
     |████████████████████████████████| 198 kB 20.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767605 sha256=6d0fbf957618196554b7632751b958eebf37e8d692e3651caf786a2a70173ec6
  Stored in directory: /root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark


In [ ]:
!pip install graphframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 9.9 MB/s 


In [ ]:
!curl -L -o "/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar" https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.1-s_2.12/graphframes-0.8.2-spark3.1-s_2.12.jar
!curl -L -o "/content/graphframes-0.8.2-spark3.1-s_2.12.jar" https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.1-s_2.12/graphframes-0.8.2-spark3.1-s_2.12.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  242k  100  242k    0     0   785k      0 --:--:-- --:--:-- --:--:--  788k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  242k  100  242k    0     0  2262k      0 --:--:-- --:--:-- --:--:-- 2262k


In [ ]:
!mkdir -p 'graphframes'
!jar -xvf '/content/graphframes-0.8.2-spark3.1-s_2.12.jar' 'graphframes'

  created: graphframes/
  created: graphframes/examples/
  created: graphframes/lib/
 inflated: graphframes/examples/__init__.py
 inflated: graphframes/tests.py
 inflated: graphframes/examples/belief_propagation.py
 inflated: graphframes/graphframe.py
 inflated: graphframes/examples/graphs.py
 inflated: graphframes/lib/aggregate_messages.py
 inflated: graphframes/lib/__init__.py
 inflated: graphframes/__init__.py
 inflated: graphframes/lib/pregel.py


## Import Libraries

In [ ]:
import pyspark

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import IntegerType
from pyspark.sql import SQLContext
from pyspark.sql.functions import sum as sqlsum
from pyspark.sql.functions import desc, asc, count
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

from graphframes.lib import AggregateMessages as AM
from graphframes import *

from functools import reduce

In [ ]:
spark = SparkSession.builder.appName('GraphFrame').getOrCreate()
spark

In [ ]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

## Example 1: create graph and basic queries

### Creating GraphFrames

In [ ]:
# Vertex DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

# Edge DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "a", "friend"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "e", "friend"),
  ("d", "a", "friend"),
  ("a", "d", "friend"),
  ("a", "e", "friend"),
  ("e", "a", "friend")], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)
g

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])

In [ ]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  a|      friend|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  e|      friend|
|  d|  a|      friend|
|  a|  d|      friend|
|  a|  e|      friend|
|  e|  a|      friend|
+---+---+------------+



In [ ]:
g.edges.filter("relationship='follow'").show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
+---+---+------------+



In [ ]:
# show pairs of users with friend relationship
g.edges.filter("relationship='friend'").show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  a|      friend|
|  e|  d|      friend|
|  d|  e|      friend|
|  d|  a|      friend|
|  a|  d|      friend|
|  a|  e|      friend|
|  e|  a|      friend|
+---+---+------------+



In [ ]:
# Find the youngset user's age the graph.
# This queries the vertex DataFrame.
g.vertices.groupBy().min('age').show()

+--------+
|min(age)|
+--------+
|      29|
+--------+



### Motif finding

In [ ]:
# Search for pairs of vertices with edges in both directions between them.
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
motifs.show()

+---------------+--------------+---------------+--------------+
|              a|             e|              b|            e2|
+---------------+--------------+---------------+--------------+
| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, d, friend}|
| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, a, friend}|
| {a, Alice, 34}|{a, b, friend}|   {b, Bob, 36}|{b, a, friend}|
| {d, David, 29}|{d, a, friend}| {a, Alice, 34}|{a, d, friend}|
|{e, Esther, 32}|{e, d, friend}| {d, David, 29}|{d, e, friend}|
| {a, Alice, 34}|{a, d, friend}| {d, David, 29}|{d, a, friend}|
|   {b, Bob, 36}|{b, a, friend}| {a, Alice, 34}|{a, b, friend}|
|{e, Esther, 32}|{e, a, friend}| {a, Alice, 34}|{a, e, friend}|
+---------------+--------------+---------------+--------------+



In [ ]:
# Vertices and edges can be anonymous
g.find("(a)-[]->(b); (b)-[]->(a)").show()

+---------------+---------------+
|              a|              b|
+---------------+---------------+
| {d, David, 29}|{e, Esther, 32}|
| {a, Alice, 34}|{e, Esther, 32}|
| {a, Alice, 34}|   {b, Bob, 36}|
| {d, David, 29}| {a, Alice, 34}|
|{e, Esther, 32}| {d, David, 29}|
| {a, Alice, 34}| {d, David, 29}|
|   {b, Bob, 36}| {a, Alice, 34}|
|{e, Esther, 32}| {a, Alice, 34}|
+---------------+---------------+



In [ ]:
# More complex queries can be expressed by applying filters
motifs.filter("b.age>30").show()

+---------------+--------------+---------------+--------------+
|              a|             e|              b|            e2|
+---------------+--------------+---------------+--------------+
| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, d, friend}|
| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, a, friend}|
| {a, Alice, 34}|{a, b, friend}|   {b, Bob, 36}|{b, a, friend}|
| {d, David, 29}|{d, a, friend}| {a, Alice, 34}|{a, d, friend}|
|   {b, Bob, 36}|{b, a, friend}| {a, Alice, 34}|{a, b, friend}|
|{e, Esther, 32}|{e, a, friend}| {a, Alice, 34}|{a, e, friend}|
+---------------+--------------+---------------+--------------+



In [ ]:
# In many cases, you only need some attribute of each vertice and egde
motifs.selectExpr("a.id as a", "b.id as b").show()

+---+---+
|  a|  b|
+---+---+
|  d|  e|
|  a|  e|
|  a|  b|
|  d|  a|
|  e|  d|
|  a|  d|
|  b|  a|
|  e|  a|
+---+---+



In [ ]:
# Find one-way relationship (i.e. follow)
g.find("(a)-[e]->(b); !(b)-[]->(a)").selectExpr("a.id", "e.relationship", "b.id").show()

+---+------------+---+
| id|relationship| id|
+---+------------+---+
|  e|      follow|  f|
|  c|      follow|  b|
|  f|      follow|  c|
+---+------------+---+



In [ ]:
chain4 = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")
chain4.show()

+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
|               a|            ab|              b|            bc|              c|            cd|              d|
+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
| {e, Esther, 32}|{e, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {a, Alice, 34}|{a, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
| {e, Esther, 32}|{e, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {d, David, 29}|{d, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|    {b, Bob, 36}|{b, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esthe

In [ ]:
# Query on sequence, with state (cnt)
#  (a) Define method for updating state given the next element of the motif.
sumFriends = lambda cnt,relationship: when(relationship == "friend", cnt+1).otherwise(cnt)
#  (b) Use sequence operation to apply method to sequence of elements in motif. In this case, the elements are the 3 edges.
condition = reduce(lambda cnt,e: sumFriends(cnt, col(e).relationship), ["ab", "bc", "cd"], lit(0))
#  (c) Apply filter to DataFrame.
chainWith2Friends2 = chain4.where(condition >= 2)
chainWith2Friends2.show()

+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
|               a|            ab|              b|            bc|              c|            cd|              d|
+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
| {e, Esther, 32}|{e, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {a, Alice, 34}|{a, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
| {e, Esther, 32}|{e, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {d, David, 29}|{d, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|    {b, Bob, 36}|{b, a, friend}| {a, Alice, 34}|{a, e, friend}|{e, Esther, 32}|{e, f, follow}| {f, Fanny, 36}|
|  {d, David, 29}|{d, e, friend}|{e, Esther, 32}|{e, d, friend}| {d, David, 29}|{d, e, friend}|{e, Esthe

### Subgraphs

In [ ]:
# Select subgraph of users older than 30, and relationships of type "friend".
# Drop isolated vertices (users) which are not contained in any edges (relationships).
g1 = g.filterVertices("age > 30").filterEdges("relationship = 'friend'").dropIsolatedVertices()

In [ ]:
g1.vertices.show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  e|Esther| 32|
|  b|   Bob| 36|
|  a| Alice| 34|
+---+------+---+



In [ ]:
g1.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  e|      friend|
|  a|  b|      friend|
|  e|  a|      friend|
|  b|  a|      friend|
+---+---+------------+



In [ ]:
# Select subgraph based on edges "e" of type "follow"
# pointing from a younger user "a" to an older user "b".
paths = g.find("(a)-[e]->(b)").filter("e.relationship = 'follow'").filter("a.age < b.age")
# "paths" contains vertex info. Extract the edges.
e2 = paths.select("e.src", "e.dst", "e.relationship")
# In Spark 1.5+, the user may simplify this call:
#  val e2 = paths.select("e.*")

# Construct the subgraph
g2 = GraphFrame(g.vertices, e2)

In [ ]:
g2.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [ ]:
g2.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  e|  f|      follow|
|  c|  b|      follow|
+---+---+------------+



### Graph algorithms

#### Breadth-first search (BFS)

In [ ]:
# Search from "Esther" for users of age < 32.
paths = g.bfs("name = 'Esther'", "age < 32")
paths.show()

# Specify edge filters or max path lengths.
paths2 = g.bfs("name = 'Esther'", "age < 32",edgeFilter="relationship != 'friend'", maxPathLength=3)
paths2.show()

+---------------+--------------+--------------+
|           from|            e0|            to|
+---------------+--------------+--------------+
|{e, Esther, 32}|{e, d, friend}|{d, David, 29}|
+---------------+--------------+--------------+

+---------------+--------------+--------------+--------------+----------------+
|           from|            e0|            v1|            e1|              to|
+---------------+--------------+--------------+--------------+----------------+
|{e, Esther, 32}|{e, f, follow}|{f, Fanny, 36}|{f, c, follow}|{c, Charlie, 30}|
+---------------+--------------+--------------+--------------+----------------+



#### Connected components

In [ ]:
# result = g.connectedComponents()
# result.select("id", "component").orderBy("component").show()

In [ ]:
# result.show()

#### Strongly connected components

In [ ]:
result = g.stronglyConnectedComponents(maxIter=10)
result.select("id", "component").orderBy("component").show()

+---+------------+
| id|   component|
+---+------------+
|  g|146028888064|
|  b|412316860416|
|  e|412316860416|
|  a|412316860416|
|  f|412316860416|
|  d|412316860416|
|  c|412316860416|
+---+------------+



#### Label Propagation Algorithm (LPA)

In [ ]:
result = g.labelPropagation(maxIter=5)
result.select("id", "label").show()

+---+-------------+
| id|        label|
+---+-------------+
|  g| 146028888064|
|  b|1460288880640|
|  e|1460288880640|
|  a|1460288880640|
|  f|1460288880640|
|  d|1460288880640|
|  c|1460288880640|
+---+-------------+



#### PageRank

In [ ]:
# Run PageRank until convergence to tolerance "tol".
results = g.pageRank(resetProbability=0.15, tol=0.01)
# Display resulting pageranks and final edge weights
# Note that the displayed pagerank may be truncated, e.g., missing the E notation.
# In Spark 1.5+, you can use show(truncate=False) to avoid truncation.
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

# Run PageRank for a fixed number of iterations.
results2 = g.pageRank(resetProbability=0.15, maxIter=10)

# Run PageRank personalized for vertex "a"
results3 = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="a")

# Run PageRank personalized for vertex ["a", "b", "c", "d"] in parallel
results4 = g.parallelPersonalizedPageRank(resetProbability=0.15, sourceIds=["a", "b", "c", "d"], maxIter=10)


+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  g|0.18618661660049637|
|  b| 1.2658584908762605|
|  e| 1.2379542381046424|
|  a| 2.0618615072239352|
|  f| 0.5341868752251782|
|  d|  1.115401191774445|
|  c| 0.5985510801950417|
+---+-------------------+

+---+---+------------------+
|src|dst|            weight|
+---+---+------------------+
|  b|  a|               1.0|
|  a|  b|0.3333333333333333|
|  d|  e|               0.5|
|  e|  a|0.3333333333333333|
|  e|  f|0.3333333333333333|
|  e|  d|0.3333333333333333|
|  a|  d|0.3333333333333333|
|  c|  b|               1.0|
|  a|  e|0.3333333333333333|
|  f|  c|               1.0|
|  d|  a|               0.5|
+---+---+------------------+



#### Shortest paths

In [ ]:
results = g.shortestPaths(landmarks=["a", "d"])
results.select("id", "distances").show()

+---+----------------+
| id|       distances|
+---+----------------+
|  g|              {}|
|  b|{a -> 1, d -> 2}|
|  e|{a -> 1, d -> 1}|
|  a|{a -> 0, d -> 1}|
|  f|{a -> 3, d -> 4}|
|  d|{a -> 1, d -> 0}|
|  c|{a -> 2, d -> 3}|
+---+----------------+



#### Triangle count

In [ ]:
results = g.triangleCount()
results.select("id", "count").show()

+---+-----+
| id|count|
+---+-----+
|  g|    0|
|  f|    0|
|  e|    1|
|  d|    1|
|  c|    0|
|  b|    0|
|  a|    1|
+---+-----+



### Saving and loading GraphFrames

In [ ]:
# Save vertices and edges as Parquet to some location.
g.vertices.write.parquet("/myLocation/vertices")
g.edges.write.parquet("/myLocation/edges")

# Load the vertices and edges back.
sameV = sqlContext.read.parquet("/myLocation/vertices")
sameE = sqlContext.read.parquet("/myLocation/edges")

# Create an identical GraphFrame.
sameG = GraphFrame(sameV, sameE)

In [ ]:
sameG.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
+---+-------+---+



### Message passing via AggregateMessages

In [ ]:
# For each user, sum the ages of the adjacent users.
msgToSrc = AM.dst["age"]
msgToDst = AM.src["age"]
agg = g.aggregateMessages(sqlsum(AM.msg).alias("summedAges"),sendToSrc=msgToSrc,sendToDst=msgToDst)
agg.show()

+---+----------+
| id|summedAges|
+---+----------+
|  f|        62|
|  e|       162|
|  d|       132|
|  c|        72|
|  b|        98|
|  a|       194|
+---+----------+



## Example 2: Graph and Bike Trip Data

In [ ]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 4 2022-2023/HỌC KỲ 1 2022 - 2023/DS104.N11 - TÍNH TOÁN SONG SONG & PHÂN TÁN/THỰC HÀNH/LAB/GraphFrame - 20 09 2022/dataset'

### Load data

In [ ]:
station = spark.read.option('header', True).csv(PATH+'/station.csv')
station.printSchema()
station.show(5)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dock_count: string (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)

+---+--------------------+------------------+-------------------+----------+--------+-----------------+
| id|                name|               lat|               long|dock_count|    city|installation_date|
+---+--------------------+------------------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|San Jose|         8/5/2013

In [ ]:
trip = spark.read.option('header', True).csv(PATH+'/trip.csv')
trip.printSchema()
trip.show(5)

root
 |-- id: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness a

In [ ]:
tripEdges = trip.withColumnRenamed('start_station_name', 'src').withColumnRenamed('end_station_name', 'dst')
tripEdges.show(5)

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|                 src|start_station_id|       end_date|                 dst|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|      70|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...|              27|8/29/2013 10:17|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|8/29/2013 11:29|  San Jose City Hall|      

## Trips From Station to Station

In [ ]:
stationGraph = GraphFrame(station, tripEdges)
stationGraph

GraphFrame(v:[id: string, name: string ... 5 more fields], e:[src: string, dst: string ... 9 more fields])

In [ ]:
topTrips = stationGraph.edges.groupBy('src', 'dst').count().orderBy(desc('count')).limit(10)
topTrips.show(truncate=False)

+----------------------------------------+----------------------------------------+-----+
|src                                     |dst                                     |count|
+----------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend) |Townsend at 7th                         |6216 |
|Harry Bridges Plaza (Ferry Building)    |Embarcadero at Sansome                  |6164 |
|Townsend at 7th                         |San Francisco Caltrain (Townsend at 4th)|5041 |
|2nd at Townsend                         |Harry Bridges Plaza (Ferry Building)    |4839 |
|Harry Bridges Plaza (Ferry Building)    |2nd at Townsend                         |4357 |
|Embarcadero at Sansome                  |Steuart at Market                       |4269 |
|Embarcadero at Folsom                   |San Francisco Caltrain (Townsend at 4th)|3967 |
|Steuart at Market                       |2nd at Townsend                         |3903 |
|2nd at So

### In Degrees and Out Degrees

In [ ]:
inDeg = stationGraph.inDegrees
inDeg.orderBy(desc('inDegree')).show(10, truncate=False)

+---------------------------------------------+--------+
|id                                           |inDegree|
+---------------------------------------------+--------+
|San Francisco Caltrain (Townsend at 4th)     |63179   |
|San Francisco Caltrain 2 (330 Townsend)      |35117   |
|Harry Bridges Plaza (Ferry Building)         |33193   |
|Embarcadero at Sansome                       |30796   |
|2nd at Townsend                              |28529   |
|Market at Sansome                            |28033   |
|Townsend at 7th                              |26637   |
|Steuart at Market                            |25025   |
|Temporary Transbay Terminal (Howard at Beale)|23080   |
|Market at 4th                                |19915   |
+---------------------------------------------+--------+
only showing top 10 rows



In [ ]:
outDeg = stationGraph.outDegrees
outDeg.orderBy(desc('outDegree')).show(10, truncate=False)

+---------------------------------------------+---------+
|id                                           |outDegree|
+---------------------------------------------+---------+
|San Francisco Caltrain (Townsend at 4th)     |49092    |
|San Francisco Caltrain 2 (330 Townsend)      |33742    |
|Harry Bridges Plaza (Ferry Building)         |32934    |
|Embarcadero at Sansome                       |27713    |
|Temporary Transbay Terminal (Howard at Beale)|26089    |
|2nd at Townsend                              |25837    |
|Steuart at Market                            |24838    |
|Market at Sansome                            |24172    |
|Townsend at 7th                              |23724    |
|Market at 10th                               |20272    |
+---------------------------------------------+---------+
only showing top 10 rows



In [ ]:
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id).drop(outDeg.id).selectExpr('id', 'double(inDegree)/double(outDegree) as degreeRatio')
degreeRatio.cache()

degreeRatio.orderBy(desc('degreeRatio')).show(10,truncate=False)

+----------------------------------------+------------------+
|id                                      |degreeRatio       |
+----------------------------------------+------------------+
|Redwood City Medical Center             |1.4533762057877813|
|Redwood City Public Library             |1.300469483568075 |
|San Francisco Caltrain (Townsend at 4th)|1.286951030717836 |
|Washington at Kearny                    |1.2723671947809878|
|MLK Library                             |1.233038348082596 |
|SJSU 4th at San Carlos                  |1.2282051282051283|
|San Mateo County Center                 |1.2195121951219512|
|Broadway at Main                        |1.208955223880597 |
|University and Emerson                  |1.2056878306878307|
|Washington at Kearney                   |1.203804347826087 |
+----------------------------------------+------------------+
only showing top 10 rows



In [ ]:
degreeRatio.orderBy(asc('degreeRatio')).show(10,truncate=False)

+-------------------------------+------------------+
|id                             |degreeRatio       |
+-------------------------------+------------------+
|Grant Avenue at Columbus Avenue|0.564700110388814 |
|2nd at Folsom                  |0.6056461731493099|
|Powell at Post (Union Square)  |0.6887003841229193|
|San Jose City Hall             |0.6928541579607188|
|San Francisco City Hall        |0.7497243660418964|
|Beale at Market                |0.774906104780699 |
|Redwood City Caltrain Station  |0.8075933075933076|
|Golden Gate at Polk            |0.8153091800599291|
|Evelyn Park and Ride           |0.8218356328734253|
|Ryland Park                    |0.8248425872925015|
+-------------------------------+------------------+
only showing top 10 rows



## Example 3: Flight data

In [ ]:
schema = StructType([StructField('id', StringType(), True),
                     StructField('fldate', StringType(), True),
                     StructField('month', IntegerType(), True),
                     StructField('dofW', IntegerType(), True),
                     StructField('carrier', StringType(), True),
                     StructField('src', StringType(), True),
                     StructField('dst', StringType(), True),
                     StructField('crsdephour', IntegerType(), True),
                     StructField('crsdeptime', IntegerType(), True),
                     StructField('depdelay', DoubleType(), True),
                     StructField('crsarrtime', IntegerType(), True),
                     StructField('arrdelay', DoubleType(), True),
                     StructField('crselapsedtime', DoubleType(), True),
                     StructField('dist', DoubleType(), True),])

In [ ]:
flightDF = spark.read.option('inferSchema', 'false').schema(schema).json(PATH+'/flightdata2018.json')
flightDF.show(5)
flightDF.count()

+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+-----+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|depdelay|crsarrtime|arrdelay|crselapsedtime| dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+-----+
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|         9|       850|     0.0|      1116|     0.0|         146.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        11|      1122|     8.0|      1349|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        14|      1356|     9.0|      1623|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        16|      1620|     0.0|      1851|     3.0|         151.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        1

282628

In [ ]:
airportVertices = spark.read.json(PATH+'/airports.json')
airportVertices.show()
airportVertices.count()

+-------------+-------+-----+---+
|         City|Country|State| id|
+-------------+-------+-----+---+
|      Chicago|    USA|   IL|ORD|
|     New York|    USA|   NY|LGA|
|       Boston|    USA|   MA|BOS|
|      Houston|    USA|   TX|IAH|
|       Newark|    USA|   NJ|EWR|
|       Denver|    USA|   CO|DEN|
|        Miami|    USA|   FL|MIA|
|San Francisco|    USA|   CA|SFO|
|      Atlanta|    USA|   GA|ATL|
|       Dallas|    USA|   TX|DFW|
|    Charlotte|    USA|   NC|CLT|
|  Los Angeles|    USA|   CA|LAX|
|      Seattle|    USA|   WA|SEA|
+-------------+-------+-----+---+



13

### Create graphframe

In [ ]:
flightGraph = GraphFrame(airportVertices, flightDF)
flightGraph

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 12 more fields])

### Querying the GraphFrame

In [ ]:
# Show vertices and edges
flightGraph.vertices.show(5)
flightGraph.edges.show(5)

+--------+-------+-----+---+
|    City|Country|State| id|
+--------+-------+-----+---+
| Chicago|    USA|   IL|ORD|
|New York|    USA|   NY|LGA|
|  Boston|    USA|   MA|BOS|
| Houston|    USA|   TX|IAH|
|  Newark|    USA|   NJ|EWR|
+--------+-------+-----+---+
only showing top 5 rows

+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+-----+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|depdelay|crsarrtime|arrdelay|crselapsedtime| dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+-----+
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|         9|       850|     0.0|      1116|     0.0|         146.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        11|      1122|     8.0|      1349|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     

In [ ]:
# Which flight routes have the longest distance?
flightGraph.edges.groupBy('src', 'dst').max('dist').sort(desc('max(dist)')).show(5)

+---+---+---------+
|src|dst|max(dist)|
+---+---+---------+
|MIA|SEA|   2724.0|
|SEA|MIA|   2724.0|
|BOS|SFO|   2704.0|
|SFO|BOS|   2704.0|
|LAX|BOS|   2611.0|
+---+---+---------+
only showing top 5 rows



In [ ]:
# Count of departure delays by Origin and destination

flightGraph.edges.filter('depdelay > 40').groupBy('src', 'dst').agg(count('depdelay').alias('flightcount')).sort(desc('flightcount')).show(5)

+---+---+-----------+
|src|dst|flightcount|
+---+---+-----------+
|ORD|LGA|        588|
|LAX|SFO|        578|
|ATL|EWR|        561|
|LGA|ORD|        532|
|ORD|SFO|        470|
+---+---+-----------+
only showing top 5 rows



In [ ]:
# What are the longest delays for flights that are greater than 1500 miles in distance?
flightGraph.edges.filter('dist > 1500').orderBy(desc('depdelay')).show(5)

+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+------+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|depdelay|crsarrtime|arrdelay|crselapsedtime|  dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+--------+----------+--------+--------------+------+
|BOS_DFW_2018-06-2...|2018-06-28|    6|   4|     AA|BOS|DFW|        10|      1012|  1345.0|      1326|  1332.0|         254.0|1562.0|
|LAX_MIA_2018-07-0...|2018-07-09|    7|   1|     AA|LAX|MIA|         9|       945|  1283.0|      1811|  1373.0|         326.0|2342.0|
|BOS_LAX_2018-03-2...|2018-03-28|    3|   3|     AA|BOS|LAX|        10|      1040|  1242.0|      1427|  1234.0|         407.0|2611.0|
|DEN_EWR_2018-07-2...|2018-07-20|    7|   5|     UA|DEN|EWR|        11|      1117|  1125.0|      1713|  1106.0|         236.0|1605.0|
|SEA_DFW_2018-07-1...|2018-07-10|    7|   2|     AA|SEA|DFW|  

In [ ]:
# What is the average delay for delayed flights departing from Boston?
flightGraph.edges.filter("src = 'BOS' and depdelay > 1").groupBy('src', 'dst').avg('depdelay').sort(desc('avg(depdelay)')).show(5)

+---+---+------------------+
|src|dst|     avg(depdelay)|
+---+---+------------------+
|BOS|EWR|57.078947368421055|
|BOS|LGA| 54.00142857142857|
|BOS|ORD|          47.97375|
|BOS|SEA| 45.56565656565657|
|BOS|DFW| 43.30031948881789|
+---+---+------------------+
only showing top 5 rows



In [ ]:
# Which airport has the most incoming flight? The most outgoing?
flightGraph.inDegrees.orderBy(desc('inDegree')).show(3)
flightGraph.outDegrees.orderBy(desc('outDegree')).show(3)

+---+--------+
| id|inDegree|
+---+--------+
|ORD|   32314|
|ATL|   30217|
|LAX|   26869|
+---+--------+
only showing top 3 rows

+---+---------+
| id|outDegree|
+---+---------+
|ORD|    32072|
|ATL|    30165|
|LAX|    26864|
+---+---------+
only showing top 3 rows



In [ ]:
# What are the hightest degree vertexes (most incoming and outgoing flight)?
flightGraph.degrees.orderBy(desc('degree')).show(3)

+---+------+
| id|degree|
+---+------+
|ORD| 64386|
|ATL| 60382|
|LAX| 53733|
+---+------+
only showing top 3 rows



In [ ]:
# What are the 3 most frequent flights in the dataset?
flightGraph.edges.groupBy('src', 'dst').count().orderBy(desc('count')).show(3)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|LGA|ORD| 4442|
|ORD|LGA| 4426|
|LAX|SFO| 4406|
+---+---+-----+
only showing top 3 rows

